### Using Larg language model & Embeddings for Q&A and Search [POC - OpenAI API demo]

Prepared for Ventera brownbag

In [7]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import os
# from utilities import GPT_utilities

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

#gptkey = GPT_utilities()
openai.api_key = "sk-"


### General example to highlight factual accuracy and confabulation i.e. 'truthiness'

In [8]:
prompt = "who won the 100 meter dash 2020 olympics?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"The 100 meter dash at the 2020 Olympics was won by Jamaica's Shelly-Ann Fraser-Pryce."

In [9]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: How many sprints in a PI?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

#### Prompt Engineering approach to mitigate accuracy and confabulation i.e. 'truthiness'

In [10]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
A Minimum Viable Product (MVP) is a version of a working product that allows the team to learn from and 
interact with their customer with the least amount of effort. MVP attends to the core customer needs 
first and as soon as possible. It helps to validate needs, reduce risk, and help the programs course correct 
quickly, as needed. Rooted in concepts that emerged from the book “The Lean Startup” by Eric Ries, the core 
idea is to facilitate a better understanding of the customers needs and interests without committing or 
using a large number of resources or fully developing a product.

Q: What's the main idea behind MVP?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'The main idea behind MVP is to facilitate a better understanding of the customers needs and interests without committing or using a large number of resources or fully developing a product.'

In [11]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
In the NFC final, the Eagles defeated the San Francisco 49ers in a 31-7 blowout. 
This will be the fourth Super Bowl appearance for the Eagles, who lost in 1980 to the Oakland Raiders and in 2004 to the New England Patriots. 
In their most recent appearance in 2017, they got their revenge with a victory over the Patriots. 
Quarterback Jalen Hurts is in just his second season as a starter for the Eagles, 
taking them to the playoffs both years.

Q: How many times have the Eagles beaten the Patriots in a superbowl?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Once'

#### Finetune Model approach to increase trustworthiness by example: preprocessing data from vstart.dev

In [12]:
df = pd.read_csv('vstart.csv')
df = df.set_index(["vscontent"])
print(f"{len(df)} rows in the data.")
df.sample(5)

45 rows in the data.


""
vscontent
"Definition of Ready is a shared understanding from the team that defines what it means for a work item to be ready to be placed in an iteration. User stories and work items can be evaluated for meeting the DoR during refinement. It is a living document determined and iteratively updated by the team, with the scrum master's guidance. Accepting a story or work item into a sprint means that the team is confident that there is enough information to deliver the work and that the user story meets the INVEST matrix (Independent, Negotiable, Valuable, Estimable, Small, Testable). The Product Owners can use DoR when preparing for user stories for upcoming sprints. The team uses it as a checklist during refinement and sprint planning to make sure that the work item is complete enough to deliver within a sprint."
"One of the challenges of beginning a new program or project is determining the most beneficial Agile Framework to manage and deliver your products. Programs that arbitrarily select a framework or are directed into a specific framework often encounter challenges managing their products and delivery. When selecting an Agile framework, several considerations are necessary, such as knowing your client, understanding their pre-existing delivery and customer landscape, and identifying their long-term delivery vision and Objectives and Key Results (OKRs). Additionally, a plan to educate your Program and teams to ensure they operate within Agile-based concepts is essential."
A Sprint Retrospective is a scrum team ceremony that occurs after the Sprint Review and before the next Sprint Planning. It is an opportunity for the Scrum Team to openly discuss what went well and what didn't in the previous sprint. The outcome is to create a plan for necessary actions and improvements to be enacted and during the next sprint.
"Post Production Validation (PPV) is the validation of an application immediately following a code push to production. This can be done for major releases or minor releases, but ideally every time something changes in production there is an external validation of the changes that have been made."
"Behavioral Driven Development (BDD) is an agile software development methodology that focuses on an application's behavior from the end user's perspective. In contrast, Test-Driven Development (TDD) focuses on testing an application's functionality. TDD does not focus on how users interact with the application, so BDD evolved from TDD."


In [13]:
def get_embedding(text: str, model: str = EMBEDDING_MODEL) -> list[float]:
    """
    Use the OpenAI Embeddings API to create an embedding for the given text.
    """
    result = openai.Embedding.create(model=model, input=text)
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[str, list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.

    Return a dictionary that maps between each embedding vector and the vscontent value of the row it corresponds to.
    """
    return {r['vscontent']: get_embedding(r['vscontent']) for idx, r in df.iterrows()}

def load_embeddings(fname: str) -> dict[str, list[float]]:
    """
    Read the document embeddings from a CSV.

    fname is the path to a CSV with exactly these named columns: 
        "vscontent", "0", "1", ... up to the length of the embedding vectors.
    """
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "vscontent"])
    return {r[0]: [r[i] for i in range(1, max_dim + 1)] for r in df.itertuples(index=False)}

df = pd.read_csv("vstart.csv")


In [14]:
document_embeddings = compute_doc_embeddings(df)

##### Lets save our embeddings in a file

In [15]:
def save_embeddings(embeddings: dict[str, list[float]], fname: str):
    df = pd.DataFrame.from_dict(embeddings, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'vscontent'}, inplace=True)
    df.to_csv(fname, index=False)


output_folder = "output"
output_fname = f"{output_folder}/embeddings.csv"
save_embeddings(document_embeddings, output_fname)

##### Lets view what an embedding looks like

In [16]:
first_text = list(document_embeddings.keys())[0]
first_embedding = list(document_embeddings.values())[0]
truncated_text = first_text[:50] + "..."
print("Example:", truncated_text, first_embedding[0])


Example: ADDIE Training Program. User training is an import... -0.012318002991378307


#####  Semantic Search use case

...by finding the most similar document embeddings to the question embedding. We're storing this locally but for a larger dataset we would consider a vector DB to index our results.

In [40]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)

    print(f"\nResponse to the question: '{query}'\nThe number represents the similarity score between a users query and\ncorresponding vSTART content for demonstration.")
    
    return document_similarities

In [18]:
# Duplicated just for demo
 
df = pd.read_csv('vstart.csv')
df = df.set_index(["vscontent"])
print(f"{len(df)} rows in the data.")
df.sample(5)

45 rows in the data.


""
vscontent
"Ideation is a creative process used to generate ideas that address a problem statement in a facilitated, judgment-free environment. The purpose of these sessions is to come up with as many ideas as possible from team members."
"Personas are fictional characters that represent actual user groups based on user research. They are created to differentiate between the distinct user groups that might use your product. The final persona artifact is a visual document that has different sections for demographic information, behaviors, goals, frustrations, motivations, and any other relevant data to that user group."
"Post Production Validation (PPV) is the validation of an application immediately following a code push to production. This can be done for major releases or minor releases, but ideally every time something changes in production there is an external validation of the changes that have been made."
"User interviews are semi-structured conversations with anyone interested in a project’s success, including users. Typically they are one-on-one sessions with a stakeholder or user to learn more about a topic."
"Whether for a short-term technical challenge or a long-term client engagement, any project needs a strong start. That's where a well-planned and conducted kickoff meeting can effectively communicate the vision, objectives, and process. Kickoffs can be geared towards the only internal team or be comprehensive and include the client. This play focuses on internal kickoff meetings; however, there are also specific tips for kickoffs involving the client."


##### Lets try a semantic search query

In [41]:
# Create a prompt for user to a enter query
uquery = input("What do you need to know?")

# Call the list of vectorized vSTART plays with nearest similarity between user query and indexed embedding. 
# Return sorted by relevance in descending order
order_document_sections_by_query_similarity(uquery, document_embeddings)[:3]


Response to the question: 'How many sprints in a PI'
The number represents the similarity score between a users query and
corresponding vSTART content for demonstration.


[(0.8116487686800455,
  "In SAFe (Scaled Agile Framework)a Program Increment (PI) typically lasts for 8 to 12 weeks and is divided into sprints. The number of sprints in a PI depends on the length of the sprint cycle that the organization has chosen.If you're not using the Scaled Agile Framework (SAFe) you can still follow the principles of Agile development and use sprints to show progress. Here are the steps to breakup your sprints.Determine the length of your sprint cycle: The length of your sprint cycle will depend on the size of your team and the complexity of your project. A common sprint cycle length is 2-4 weeks.Identify your sprint backlog: This is a prioritized list of tasks that your team will work on during the sprint. The sprint backlog should be updated regularly to reflect changes in the project requirements and priorities.Plan your sprint: During the sprint planning meeting your team will review the sprint backlog and determine which tasks they can complete during the s